# Hidden Markov Model - Forward - Backward Algorithm


In [1]:
# Import libraries
import numpy as np
import pandas as pd
from collections import namedtuple
from itertools import permutations, chain, product
from sklearn.cluster import KMeans
from inspect import Signature, Parameter

In [2]:
CORN_2013_2017 = './Asgn11_data/datasets_corn2013-2017.txt'

In [3]:
corn13_17 = pd.read_csv(CORN_2013_2017, names = ("week","price") )

In [4]:
def generate_cluster_assignments(ser, clusters):
    
    clusterer = KMeans(n_clusters=clusters,random_state=24)
    df=pd.DataFrame(ser)
    clusterer.fit(df)
      
    return clusterer.predict(df) 

In [5]:
corn13_17_seq = generate_cluster_assignments(corn13_17[['price']], 5)
seq=corn13_17_seq
seq

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 3, 3, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4])

# Defining Matrices

In [6]:
#pi:initial distribution
pi=np.array([[0.4],[0.6]])

In [7]:
#State emission
B=np.array([[0.1,0.3, 0.4, 0.15,0.05],[0.15,0.2, 0.3, 0.05,0.3]])
B

array([[0.1 , 0.3 , 0.4 , 0.15, 0.05],
       [0.15, 0.2 , 0.3 , 0.05, 0.3 ]])

In [8]:
#State transition
#A=[[S1S1,S1S2],[S2S1,S2S2]], 0.55 is wrong, but wanted to get same results as class example
A=np.array([[0.4,0.6],[0.35,0.55]])
A

array([[0.4 , 0.6 ],
       [0.35, 0.55]])

# STEP 1: ESTIMATE PROBABILITIES:

In [9]:
def calculate_probabilities(sequence, A, B, pi):
    obs_prob=np.zeros((len(sequence)-1,4))
    highest_probs=np.zeros((len(sequence)-1,1))
    highest_states=np.zeros((len(sequence)-1,2))
    highest_prob = None
    for idx, obs in enumerate(sequence):
        if idx != 0: 
            prob_lst=[]
            for (a1,a2), value in np.ndenumerate(A):
                prev_prob = pi[a1] * B[a1][sequence[idx-1]]
                curr_prob=  A[a1][a2]*B[a2][obs]
                combined_prob=np.round_(prev_prob*curr_prob,4)
                prob_lst.append(combined_prob)
                if highest_prob is None or combined_prob > highest_prob:
                    highest_prob=combined_prob
                    highest_trans=a1,a2
            obs_prob[idx-1]=prob_lst
            highest_probs[idx-1]=highest_prob
            highest_states[idx-1][0]=highest_trans[0]
            highest_states[idx-1][1]=highest_trans[1]
        else:
            continue
    obs_probs=np.concatenate((obs_prob, highest_probs,highest_states), axis=1)
    obs_probs=np.around(obs_probs,3)
    print(obs_probs)
    return (obs_prob, A, B, pi)    

In [10]:
obs_prob, A, B, pi=calculate_probabilities(corn13_17_seq, A, B, pi)

[[0.014 0.014 0.013 ... 0.014 0.    0.   ]
 [0.014 0.014 0.013 ... 0.014 0.    0.   ]
 [0.014 0.014 0.013 ... 0.014 0.    0.   ]
 ...
 [0.    0.004 0.003 ... 0.03  1.    1.   ]
 [0.    0.004 0.003 ... 0.03  1.    1.   ]
 [0.    0.004 0.003 ... 0.03  1.    1.   ]]


# STEP 2: UPDATE PARAMETERS

In [11]:
def observed_pairs(sequence):
    observed_pairs = []
    for idx in range(len(sequence)-1):
        observed_pairs.append((sequence[idx], sequence[idx+1]))
    return observed_pairs

In [12]:
seq=corn13_17_seq #repeated mandatory

In [13]:
obs_pairs = observed_pairs(seq)

In [14]:
def emission_matrix_update(sequence, A, B, pi): #provided, altered
    new_B = np.zeros((len(A),B.shape[1]))
    for obs in np.unique(sequence):
        #print(obs)
        inc_seq = [seq for seq in obs_pairs if seq.count(obs)>0]
        highest_pairs = []
        for seq in inc_seq:
            prob_pairs = []
            for (a1,a2), value in np.ndenumerate(A):
                obs1, obs2 = seq
                prob = pi[a1] * B[a1][obs1]
                prob *= A[a1][a2]*B[a2][obs2]
                prob = np.round_(prob,5)
                assoc_tuples = [(a1, obs1),(a2, obs2)]
                prob_pairs.append([assoc_tuples, prob])
            prob_pairs = sorted(prob_pairs, key = lambda x: x[1], reverse = True)
            to_add = {'highest':prob_pairs[0][1]}
            for i in range(len(A)):
                highest_of_state = 0
                #find first pair
                for pp in prob_pairs:
                    if pp[0].count((i,obs))>0:
                        highest_of_state = pp[1]
                        break
                to_add[i] = highest_of_state
            highest_pairs.append(to_add)
        highest_probability =sum([d['highest'] for d in highest_pairs])
    #     print(highest_probability)
    #     print(highest_pairs)
        for i in range(len(A)):
            new_B[i][obs]= sum([d[i] for d in highest_pairs])/highest_probability   
    return new_B
        

In [15]:
nb = emission_matrix_update(corn13_17_seq, A,B,pi)
nb

array([[0.48402187, 0.99858156, 0.96760259, 1.        , 0.12109206],
       [1.        , 0.99148936, 0.99990183, 0.49800818, 1.        ]])

In [16]:
def normB(x):
        return [n/np.sum(x) for n in x]

In [17]:
new_B=np.apply_along_axis(normB,1,nb)
new_B

array([[0.13553108, 0.27961305, 0.27093862, 0.28001023, 0.03390701],
       [0.22274695, 0.22085123, 0.22272508, 0.1109298 , 0.22274695]])

In [18]:
def new_state_trans(probabilities):
    highest_prob_sum = 0
    def sum(x):
        return np.sum(x)
    for obs in probabilities:
        highest_prob_sum += np.amax(obs)
    obs_out=np.apply_along_axis(sum, 0, probabilities)
    new_A=obs_out.reshape(2,-1)
    new_A=new_A/highest_prob_sum
    count=0
    for row in new_A:    
        row_sum=np.sum(row)
        row=row/row_sum
        new_A[count]=row  
        count+=1
    return new_A
        
  


In [19]:
new_A = new_state_trans(obs_prob)
new_A

array([[0.45133927, 0.54866073],
       [0.32809729, 0.67190271]])

# STEP 3: REPEAT UNTIL PARAMETERS CONVERGE

In [20]:
ob_prob2, A2, B2, pi2 = calculate_probabilities(corn13_17_seq, new_A, new_B, pi)

[[0.014 0.014 0.012 ... 0.02  1.    1.   ]
 [0.014 0.014 0.012 ... 0.02  1.    1.   ]
 [0.014 0.014 0.012 ... 0.02  1.    1.   ]
 ...
 [0.    0.002 0.002 ... 0.02  1.    1.   ]
 [0.    0.002 0.002 ... 0.02  1.    1.   ]
 [0.    0.002 0.002 ... 0.02  1.    1.   ]]


In [21]:
new_A2=new_state_trans(ob_prob2)

In [22]:
nb2 = emission_matrix_update(corn13_17_seq, A2, B2, pi2)
new_B2=np.apply_along_axis(normB,1,nb2)
new_B2

array([[0.12283468, 0.25421973, 0.23776507, 0.35570363, 0.02947689],
       [0.22102679, 0.22102679, 0.22072879, 0.11619083, 0.22102679]])